In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
np.set_printoptions(precision=4)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import statsmodels.api as sms
presnapoffense = ['pff_QUARTER','pff_DOWN','pff_DRIVE','pff_OFFFORMATIONUNBALANCED','pff_RUNPASSOPTION',
                  'isshotgun','isemptybackfield','teleft','teright','teboth','numberwidereceivers','numberrunningbacks','numbertightends','numberwrleft',
                 'numberwrright','numberpassblocking','blitz']

# This part makes it so that you only look at the pass plays
df = pd.read_csv('FinalTCUTrain.csv')
df=df[df.pff_NOPLAY ==0]
df=df[df["pff_PENALTYYARDS"].isnull()]
df.pff_OFFPERSONNELBASIC.dropna()
df.pff_OFFPERSONNEL.dropna()
df.pff_DEFPERSONNEL.dropna()
df.pff_GAINLOSS.fillna(0,inplace=True)
df['shownisclosed'] = np.where((df['pff_MOFOCSHOWN'] == "C") , 1, 0)
df['playedisclosed'] = np.where((df['pff_MOFOCPLAYED'] == "C") , 1, 0)
df['isman'] = np.where((df['pff_PASSCOVERAGE']=="0")|(df['pff_PASSCOVERAGE']=='1')|(df['pff_PASSCOVERAGE']=='1D')|(df['pff_PASSCOVERAGE']=='2M')|(df['pff_PASSCOVERAGE']=='GL') , 1, 0)
df['hashposition'] = df['pff_HASH']
df.hashposition=df.hashposition.replace("L",0)
df.hashposition=df.hashposition.replace("C",1)
df.hashposition=df.hashposition.replace("R",2)
df['ispress'] = np.where((df['pff_PRESS'].isna()) , 0, 1)
df['isshotgun'] = np.where((df['pff_SHOTGUN'].isna()) , 0, 1)
df['isemptybackfield'] = np.where((df['pff_RBALIGNMENT']=="EMPTY") , 1, 0)
df['isrush'] = np.where((df['pff_RUNPASS'] == 'R'), 1, 0)
df['teleft'] = np.where((df['pff_TEALIGNMENT'].str[0]=="L")|(df['pff_TEALIGNMENT'].str[2]=="L")|(df['pff_TEALIGNMENT'].str[4]=="L")|(df['pff_TEALIGNMENT'].str[6]=="L") , 1, 0)
df['teright'] = np.where((df['pff_TEALIGNMENT'].str[0]=="R")|(df['pff_TEALIGNMENT'].str[2]=="R")|(df['pff_TEALIGNMENT'].str[4]=="R")|(df['pff_TEALIGNMENT'].str[6]=="R") , 1, 0)
df['teboth']= np.where((df['teleft']== 1)&(df['teright']==1),1,0)
df = df[df.pff_DEFPERSONNEL != "10 Men"]
df = df[df.pff_OFFPERSONNEL != "10 Men"]
df['numberlinemen']=df['pff_DEFPERSONNEL'].astype(str).str[0]
df['numberlinebackers']=df['pff_DEFPERSONNEL'].astype(str).str[2]
df['numbersecondary']=df['pff_DEFPERSONNEL'].astype(str).str[4]
df['numberwidereceivers']=df['pff_OFFPERSONNEL'].astype(str).str[0]
df['numberrunningbacks']=df['pff_OFFPERSONNEL'].astype(str).str[4]
df['numbertightends']=df['pff_OFFPERSONNEL'].astype(str).str[6]
df['numberrushers']=df['pff_PASSRUSHPLAYERS'].astype(str).str[0]
df['numbercoverage']=df['pff_PASSCOVERAGEPLAYERS'].astype(str).str[0]
df['numberwrleft']=df['pff_OFFFORMATIONGROUP'].astype(str).str[0]
df['numberwrright']=df['pff_OFFFORMATIONGROUP'].astype(str).str[2]
df['numberpassblocking']=df['pff_PASSBLOCKING'].astype(str).str[0]
df['numberboxplayers']=df['pff_BOXPLAYERS'].astype(str).str[0]
df=df[pd.to_numeric(df['numberlinemen'], errors='coerce').notnull()]
df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
df = df[df.numberwidereceivers != "X"]
df = df[df.numberrunningbacks != "X"]
df = df[df.numbertightends != "X"]
df.numberrushers=df.numberrushers.replace("n",0)
df.numberpassblocking=df.numberpassblocking.replace("n",0)
df.numbercoverage=df.numbercoverage.replace("n",0)
df['pff_DRIVE'] = df['pff_DRIVE'].astype(np.int64)
df['pff_DRIVEPLAY'] = df['pff_DRIVEPLAY'].astype(np.int64)
df['pff_GAINLOSS'] = df['pff_GAINLOSS'].astype(np.int64)
df['numberlinemen'] = df['numberlinemen'].astype(np.int64)
df['numberlinebackers'] = df['numberlinebackers'].astype(np.int64)
df['numbersecondary'] = df['numbersecondary'].astype(np.int64)
df['numberwidereceivers'] = df['numberwidereceivers'].astype(np.int64)
df['numberrunningbacks'] = df['numberrunningbacks'].astype(np.int64)
df['numbertightends'] = df['numbertightends'].astype(np.int64)
df['numberrushers'] = df['numberrushers'].astype(np.int64)
df['numbercoverage'] = df['numbercoverage'].astype(np.int64)
df['numberwrleft'] = df['numberwrleft'].astype(np.int64)
df['numberwrright'] = df['numberwrright'].astype(np.int64)
df['numberpassblocking'] = df['numberpassblocking'].astype(np.int64)
df['numberboxplayers'] = df['numberboxplayers'].astype(np.int64)
df['fieldposition'] = np.where((df['pff_FIELDPOSITION'] >= 0),df['pff_FIELDPOSITION'] , df['pff_FIELDPOSITION']+100)
df['efficent'] = np.where((df['pff_DOWN'] == 1) & (df['pff_DISTANCE']*.4 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 2) & (df['pff_DISTANCE']*.5 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 3) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 4) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS']), 1, 0)
df['redzone'] = np.where((df['pff_FIELDPOSITION'] <= 20), 1, 0)
df['backedup'] = np.where((df['pff_FIELDPOSITION'] >= 80) , 1, 0)
df['blitz'] = np.where((df['numberrushers'] > 4), 1, 0)
df = df[df.numberrushers !=0]
TCUdata = df.loc[:,presnapoffense]
#--------------------------------------------------------------------------------------------------------------------------------------------------------
df = pd.read_csv('FinalTCUTest.csv')
df=df[df.pff_NOPLAY ==0]
df=df[df["pff_PENALTYYARDS"].isnull()]
df.pff_OFFPERSONNELBASIC.dropna()
df.pff_OFFPERSONNEL.dropna()
df.pff_DEFPERSONNEL.dropna()
df.pff_GAINLOSS.fillna(0,inplace=True)
df['shownisclosed'] = np.where((df['pff_MOFOCSHOWN'] == "C") , 1, 0)
df['playedisclosed'] = np.where((df['pff_MOFOCPLAYED'] == "C") , 1, 0)
df['isman'] = np.where((df['pff_PASSCOVERAGE']=="0")|(df['pff_PASSCOVERAGE']=='1')|(df['pff_PASSCOVERAGE']=='1D')|(df['pff_PASSCOVERAGE']=='2M')|(df['pff_PASSCOVERAGE']=='GL') , 1, 0)
df['hashposition'] = df['pff_HASH']
df.hashposition=df.hashposition.replace("L",0)
df.hashposition=df.hashposition.replace("C",1)
df.hashposition=df.hashposition.replace("R",2)
df['ispress'] = np.where((df['pff_PRESS'].isna()) , 0, 1)
df['isshotgun'] = np.where((df['pff_SHOTGUN'].isna()) , 0, 1)
df['isemptybackfield'] = np.where((df['pff_RBALIGNMENT']=="EMPTY") , 1, 0)
df['isrush'] = np.where((df['pff_RUNPASS'] == 'R'), 1, 0)
df['teleft'] = np.where((df['pff_TEALIGNMENT'].str[0]=="L")|(df['pff_TEALIGNMENT'].str[2]=="L")|(df['pff_TEALIGNMENT'].str[4]=="L")|(df['pff_TEALIGNMENT'].str[6]=="L") , 1, 0)
df['teright'] = np.where((df['pff_TEALIGNMENT'].str[0]=="R")|(df['pff_TEALIGNMENT'].str[2]=="R")|(df['pff_TEALIGNMENT'].str[4]=="R")|(df['pff_TEALIGNMENT'].str[6]=="R") , 1, 0)
df['teboth']= np.where((df['teleft']== 1)&(df['teright']==1),1,0)
df = df[df.pff_DEFPERSONNEL != "10 Men"]
df = df[df.pff_OFFPERSONNEL != "10 Men"]
df['numberlinemen']=df['pff_DEFPERSONNEL'].astype(str).str[0]
df['numberlinebackers']=df['pff_DEFPERSONNEL'].astype(str).str[2]
df['numbersecondary']=df['pff_DEFPERSONNEL'].astype(str).str[4]
df['numberwidereceivers']=df['pff_OFFPERSONNEL'].astype(str).str[0]
df['numberrunningbacks']=df['pff_OFFPERSONNEL'].astype(str).str[4]
df['numbertightends']=df['pff_OFFPERSONNEL'].astype(str).str[6]
df['numberrushers']=df['pff_PASSRUSHPLAYERS'].astype(str).str[0]
df['numbercoverage']=df['pff_PASSCOVERAGEPLAYERS'].astype(str).str[0]
df['numberwrleft']=df['pff_OFFFORMATIONGROUP'].astype(str).str[0]
df['numberwrright']=df['pff_OFFFORMATIONGROUP'].astype(str).str[2]
df['numberpassblocking']=df['pff_PASSBLOCKING'].astype(str).str[0]
df['numberboxplayers']=df['pff_BOXPLAYERS'].astype(str).str[0]
df=df[pd.to_numeric(df['numberlinemen'], errors='coerce').notnull()]
df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
df = df[df.numberwidereceivers != "X"]
df = df[df.numberrunningbacks != "X"]
df = df[df.numbertightends != "X"]
df.numberrushers=df.numberrushers.replace("n",0)
df.numberpassblocking=df.numberpassblocking.replace("n",0)
df.numbercoverage=df.numbercoverage.replace("n",0)
df['pff_DRIVE'] = df['pff_DRIVE'].astype(np.int64)
df['pff_DRIVEPLAY'] = df['pff_DRIVEPLAY'].astype(np.int64)
df['pff_GAINLOSS'] = df['pff_GAINLOSS'].astype(np.int64)
df['numberlinemen'] = df['numberlinemen'].astype(np.int64)
df['numberlinebackers'] = df['numberlinebackers'].astype(np.int64)
df['numbersecondary'] = df['numbersecondary'].astype(np.int64)
df['numberwidereceivers'] = df['numberwidereceivers'].astype(np.int64)
df['numberrunningbacks'] = df['numberrunningbacks'].astype(np.int64)
df['numbertightends'] = df['numbertightends'].astype(np.int64)
df['numberrushers'] = df['numberrushers'].astype(np.int64)
df['numbercoverage'] = df['numbercoverage'].astype(np.int64)
df['numberwrleft'] = df['numberwrleft'].astype(np.int64)
df['numberwrright'] = df['numberwrright'].astype(np.int64)
df['numberpassblocking'] = df['numberpassblocking'].astype(np.int64)
df['numberboxplayers'] = df['numberboxplayers'].astype(np.int64)
df['fieldposition'] = np.where((df['pff_FIELDPOSITION'] >= 0),df['pff_FIELDPOSITION'] , df['pff_FIELDPOSITION']+100)
df['efficent'] = np.where((df['pff_DOWN'] == 1) & (df['pff_DISTANCE']*.4 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 2) & (df['pff_DISTANCE']*.5 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 3) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 4) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS']), 1, 0)
df['redzone'] = np.where((df['pff_FIELDPOSITION'] <= 20), 1, 0)
df['backedup'] = np.where((df['pff_FIELDPOSITION'] >= 80) , 1, 0)
df['blitz'] = np.where((df['numberrushers'] > 4), 1, 0)
df = df[df.numberrushers !=0]
TCUtestdata = df.loc[:,presnapoffense]

# Creates the model
X_train, X_test = TCUdata.drop('blitz', axis=1), TCUtestdata.drop('blitz', axis=1)
y_train, y_test = TCUdata['blitz'], TCUtestdata['blitz']
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)
y_pred = logmodel.predict(X_test)
print('-------------------------TCU MODEL-----------------------------')
print('Classification Report:')
print('')
print(classification_report(y_test, y_pred))
print('MODEL ACCURACY:',classification_report(y_test, y_pred)[205:207],'%')
# Prints confusion matrix
print('------------------------------------------------------')
print('Confusion Matrix:')
print('')
print(confusion_matrix(y_test, y_pred))

-------------------------TCU MODEL-----------------------------
Classification Report:

              precision    recall  f1-score   support

           0       0.77      0.94      0.85        18
           1       0.00      0.00      0.00         5

    accuracy                           0.74        23
   macro avg       0.39      0.47      0.42        23
weighted avg       0.60      0.74      0.67        23

MODEL ACCURACY: 74 %
------------------------------------------------------
Confusion Matrix:

[[17  1]
 [ 5  0]]


C:\Users\2tann\AppData\Local\Temp\ipykernel_19216\3612813558.py:60: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
C:\Users\2tann\AppData\Local\Temp\ipykernel_19216\3612813558.py:129: FutureWarning: In a future version of pandas

In [7]:
# This part makes it so that you only look at the pass plays
df = pd.read_csv('FinalBAYTrain.csv')
df=df[df.pff_NOPLAY ==0]
df=df[df["pff_PENALTYYARDS"].isnull()]
df.pff_OFFPERSONNELBASIC.dropna()
df.pff_OFFPERSONNEL.dropna()
df.pff_DEFPERSONNEL.dropna()
df.pff_GAINLOSS.fillna(0,inplace=True)
df['shownisclosed'] = np.where((df['pff_MOFOCSHOWN'] == "C") , 1, 0)
df['playedisclosed'] = np.where((df['pff_MOFOCPLAYED'] == "C") , 1, 0)
df['isman'] = np.where((df['pff_PASSCOVERAGE']=="0")|(df['pff_PASSCOVERAGE']=='1')|(df['pff_PASSCOVERAGE']=='1D')|(df['pff_PASSCOVERAGE']=='2M')|(df['pff_PASSCOVERAGE']=='GL') , 1, 0)
df['hashposition'] = df['pff_HASH']
df.hashposition=df.hashposition.replace("L",0)
df.hashposition=df.hashposition.replace("C",1)
df.hashposition=df.hashposition.replace("R",2)
df['ispress'] = np.where((df['pff_PRESS'].isna()) , 0, 1)
df['isshotgun'] = np.where((df['pff_SHOTGUN'].isna()) , 0, 1)
df['isemptybackfield'] = np.where((df['pff_RBALIGNMENT']=="EMPTY") , 1, 0)
df['isrush'] = np.where((df['pff_RUNPASS'] == 'R'), 1, 0)
df['teleft'] = np.where((df['pff_TEALIGNMENT'].str[0]=="L")|(df['pff_TEALIGNMENT'].str[2]=="L")|(df['pff_TEALIGNMENT'].str[4]=="L")|(df['pff_TEALIGNMENT'].str[6]=="L") , 1, 0)
df['teright'] = np.where((df['pff_TEALIGNMENT'].str[0]=="R")|(df['pff_TEALIGNMENT'].str[2]=="R")|(df['pff_TEALIGNMENT'].str[4]=="R")|(df['pff_TEALIGNMENT'].str[6]=="R") , 1, 0)
df['teboth']= np.where((df['teleft']== 1)&(df['teright']==1),1,0)
df = df[df.pff_DEFPERSONNEL != "10 Men"]
df = df[df.pff_OFFPERSONNEL != "10 Men"]
df['numberlinemen']=df['pff_DEFPERSONNEL'].astype(str).str[0]
df['numberlinebackers']=df['pff_DEFPERSONNEL'].astype(str).str[2]
df['numbersecondary']=df['pff_DEFPERSONNEL'].astype(str).str[4]
df['numberwidereceivers']=df['pff_OFFPERSONNEL'].astype(str).str[0]
df['numberrunningbacks']=df['pff_OFFPERSONNEL'].astype(str).str[4]
df['numbertightends']=df['pff_OFFPERSONNEL'].astype(str).str[6]
df['numberrushers']=df['pff_PASSRUSHPLAYERS'].astype(str).str[0]
df['numbercoverage']=df['pff_PASSCOVERAGEPLAYERS'].astype(str).str[0]
df['numberwrleft']=df['pff_OFFFORMATIONGROUP'].astype(str).str[0]
df['numberwrright']=df['pff_OFFFORMATIONGROUP'].astype(str).str[2]
df['numberpassblocking']=df['pff_PASSBLOCKING'].astype(str).str[0]
df['numberboxplayers']=df['pff_BOXPLAYERS'].astype(str).str[0]
df=df[pd.to_numeric(df['numberlinemen'], errors='coerce').notnull()]
df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
df = df[df.numberwidereceivers != "X"]
df = df[df.numberrunningbacks != "X"]
df = df[df.numbertightends != "X"]
df.numberrushers=df.numberrushers.replace("n",0)
df.numberpassblocking=df.numberpassblocking.replace("n",0)
df.numbercoverage=df.numbercoverage.replace("n",0)
df['pff_DRIVE'] = df['pff_DRIVE'].astype(np.int64)
df['pff_DRIVEPLAY'] = df['pff_DRIVEPLAY'].astype(np.int64)
df['pff_GAINLOSS'] = df['pff_GAINLOSS'].astype(np.int64)
df['numberlinemen'] = df['numberlinemen'].astype(np.int64)
df['numberlinebackers'] = df['numberlinebackers'].astype(np.int64)
df['numbersecondary'] = df['numbersecondary'].astype(np.int64)
df['numberwidereceivers'] = df['numberwidereceivers'].astype(np.int64)
df['numberrunningbacks'] = df['numberrunningbacks'].astype(np.int64)
df['numbertightends'] = df['numbertightends'].astype(np.int64)
df['numberrushers'] = df['numberrushers'].astype(np.int64)
df['numbercoverage'] = df['numbercoverage'].astype(np.int64)
df['numberwrleft'] = df['numberwrleft'].astype(np.int64)
df['numberwrright'] = df['numberwrright'].astype(np.int64)
df['numberpassblocking'] = df['numberpassblocking'].astype(np.int64)
df['numberboxplayers'] = df['numberboxplayers'].astype(np.int64)
df['fieldposition'] = np.where((df['pff_FIELDPOSITION'] >= 0),df['pff_FIELDPOSITION'] , df['pff_FIELDPOSITION']+100)
df['efficent'] = np.where((df['pff_DOWN'] == 1) & (df['pff_DISTANCE']*.4 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 2) & (df['pff_DISTANCE']*.5 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 3) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 4) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS']), 1, 0)
df['redzone'] = np.where((df['pff_FIELDPOSITION'] <= 20), 1, 0)
df['backedup'] = np.where((df['pff_FIELDPOSITION'] >= 80) , 1, 0)
df['blitz'] = np.where((df['numberrushers'] > 4), 1, 0)
df = df[df.numberrushers !=0]
Baylordata = df.loc[:,presnapoffense]
#-------------------------------------------------------------------------------------------------------------------------------
df = pd.read_csv('FinalBAYTest.csv')
df=df[df.pff_NOPLAY ==0]
df=df[df["pff_PENALTYYARDS"].isnull()]
df.pff_OFFPERSONNELBASIC.dropna()
df.pff_OFFPERSONNEL.dropna()
df.pff_DEFPERSONNEL.dropna()
df.pff_GAINLOSS.fillna(0,inplace=True)
df['shownisclosed'] = np.where((df['pff_MOFOCSHOWN'] == "C") , 1, 0)
df['playedisclosed'] = np.where((df['pff_MOFOCPLAYED'] == "C") , 1, 0)
df['isman'] = np.where((df['pff_PASSCOVERAGE']=="0")|(df['pff_PASSCOVERAGE']=='1')|(df['pff_PASSCOVERAGE']=='1D')|(df['pff_PASSCOVERAGE']=='2M')|(df['pff_PASSCOVERAGE']=='GL') , 1, 0)
df['hashposition'] = df['pff_HASH']
df.hashposition=df.hashposition.replace("L",0)
df.hashposition=df.hashposition.replace("C",1)
df.hashposition=df.hashposition.replace("R",2)
df['ispress'] = np.where((df['pff_PRESS'].isna()) , 0, 1)
df['isshotgun'] = np.where((df['pff_SHOTGUN'].isna()) , 0, 1)
df['isemptybackfield'] = np.where((df['pff_RBALIGNMENT']=="EMPTY") , 1, 0)
df['isrush'] = np.where((df['pff_RUNPASS'] == 'R'), 1, 0)
df['teleft'] = np.where((df['pff_TEALIGNMENT'].str[0]=="L")|(df['pff_TEALIGNMENT'].str[2]=="L")|(df['pff_TEALIGNMENT'].str[4]=="L")|(df['pff_TEALIGNMENT'].str[6]=="L") , 1, 0)
df['teright'] = np.where((df['pff_TEALIGNMENT'].str[0]=="R")|(df['pff_TEALIGNMENT'].str[2]=="R")|(df['pff_TEALIGNMENT'].str[4]=="R")|(df['pff_TEALIGNMENT'].str[6]=="R") , 1, 0)
df['teboth']= np.where((df['teleft']== 1)&(df['teright']==1),1,0)
df = df[df.pff_DEFPERSONNEL != "10 Men"]
df = df[df.pff_OFFPERSONNEL != "10 Men"]
df['numberlinemen']=df['pff_DEFPERSONNEL'].astype(str).str[0]
df['numberlinebackers']=df['pff_DEFPERSONNEL'].astype(str).str[2]
df['numbersecondary']=df['pff_DEFPERSONNEL'].astype(str).str[4]
df['numberwidereceivers']=df['pff_OFFPERSONNEL'].astype(str).str[0]
df['numberrunningbacks']=df['pff_OFFPERSONNEL'].astype(str).str[4]
df['numbertightends']=df['pff_OFFPERSONNEL'].astype(str).str[6]
df['numberrushers']=df['pff_PASSRUSHPLAYERS'].astype(str).str[0]
df['numbercoverage']=df['pff_PASSCOVERAGEPLAYERS'].astype(str).str[0]
df['numberwrleft']=df['pff_OFFFORMATIONGROUP'].astype(str).str[0]
df['numberwrright']=df['pff_OFFFORMATIONGROUP'].astype(str).str[2]
df['numberpassblocking']=df['pff_PASSBLOCKING'].astype(str).str[0]
df['numberboxplayers']=df['pff_BOXPLAYERS'].astype(str).str[0]
df=df[pd.to_numeric(df['numberlinemen'], errors='coerce').notnull()]
df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
df = df[df.numberwidereceivers != "X"]
df = df[df.numberrunningbacks != "X"]
df = df[df.numbertightends != "X"]
df.numberrushers=df.numberrushers.replace("n",0)
df.numberpassblocking=df.numberpassblocking.replace("n",0)
df.numbercoverage=df.numbercoverage.replace("n",0)
df['pff_DRIVE'] = df['pff_DRIVE'].astype(np.int64)
df['pff_DRIVEPLAY'] = df['pff_DRIVEPLAY'].astype(np.int64)
df['pff_GAINLOSS'] = df['pff_GAINLOSS'].astype(np.int64)
df['numberlinemen'] = df['numberlinemen'].astype(np.int64)
df['numberlinebackers'] = df['numberlinebackers'].astype(np.int64)
df['numbersecondary'] = df['numbersecondary'].astype(np.int64)
df['numberwidereceivers'] = df['numberwidereceivers'].astype(np.int64)
df['numberrunningbacks'] = df['numberrunningbacks'].astype(np.int64)
df['numbertightends'] = df['numbertightends'].astype(np.int64)
df['numberrushers'] = df['numberrushers'].astype(np.int64)
df['numbercoverage'] = df['numbercoverage'].astype(np.int64)
df['numberwrleft'] = df['numberwrleft'].astype(np.int64)
df['numberwrright'] = df['numberwrright'].astype(np.int64)
df['numberpassblocking'] = df['numberpassblocking'].astype(np.int64)
df['numberboxplayers'] = df['numberboxplayers'].astype(np.int64)
df['fieldposition'] = np.where((df['pff_FIELDPOSITION'] >= 0),df['pff_FIELDPOSITION'] , df['pff_FIELDPOSITION']+100)
df['efficent'] = np.where((df['pff_DOWN'] == 1) & (df['pff_DISTANCE']*.4 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 2) & (df['pff_DISTANCE']*.5 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 3) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 4) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS']), 1, 0)
df['redzone'] = np.where((df['pff_FIELDPOSITION'] <= 20), 1, 0)
df['backedup'] = np.where((df['pff_FIELDPOSITION'] >= 80) , 1, 0)
df['blitz'] = np.where((df['numberrushers'] > 4), 1, 0)
df = df[df.numberrushers !=0]
Baylortestdata = df.loc[:,presnapoffense]

# Creates the model
BAYX_train, BAYX_test = Baylordata.drop('blitz', axis=1), Baylortestdata.drop('blitz', axis=1)
BAYy_train, BAYy_test = Baylordata['blitz'], Baylortestdata['blitz']
BAYlogmodel = LogisticRegression()
BAYlogmodel.fit(BAYX_train, BAYy_train)
BAYy_pred = logmodel.predict(BAYX_test)
print('-------------------------BAYLOR MODEL-----------------------------')
print('Classification Report:')
print('')
print(classification_report(BAYy_test, BAYy_pred))
print('MODEL ACCURACY:',classification_report(BAYy_test, BAYy_pred)[205:207],'%')
print('------------------------------------------------------')
# Prints confusion matrix
print('Confusion Matrix:')
print('')
print(confusion_matrix(BAYy_test, BAYy_pred))

-------------------------BAYLOR MODEL-----------------------------
Classification Report:

              precision    recall  f1-score   support

           0       0.79      1.00      0.89        27
           1       1.00      0.12      0.22         8

    accuracy                           0.80        35
   macro avg       0.90      0.56      0.55        35
weighted avg       0.84      0.80      0.73        35

MODEL ACCURACY: 80 %
------------------------------------------------------
Confusion Matrix:

[[27  0]
 [ 7  1]]


C:\Users\2tann\AppData\Local\Temp\ipykernel_19216\958730658.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
C:\Users\2tann\AppData\Local\Temp\ipykernel_19216\958730658.py:107: FutureWarning: In a future version of pandas a

In [8]:
# This part makes it so that you only look at the pass plays
df = pd.read_csv('FinalKUTrain.csv')
df=df[df.pff_NOPLAY ==0]
df=df[df["pff_PENALTYYARDS"].isnull()]
df.pff_OFFPERSONNELBASIC.dropna()
df.pff_OFFPERSONNEL.dropna()
df.pff_DEFPERSONNEL.dropna()
df.pff_GAINLOSS.fillna(0,inplace=True)
df['shownisclosed'] = np.where((df['pff_MOFOCSHOWN'] == "C") , 1, 0)
df['playedisclosed'] = np.where((df['pff_MOFOCPLAYED'] == "C") , 1, 0)
df['isman'] = np.where((df['pff_PASSCOVERAGE']=="0")|(df['pff_PASSCOVERAGE']=='1')|(df['pff_PASSCOVERAGE']=='1D')|(df['pff_PASSCOVERAGE']=='2M')|(df['pff_PASSCOVERAGE']=='GL') , 1, 0)
df['hashposition'] = df['pff_HASH']
df.hashposition=df.hashposition.replace("L",0)
df.hashposition=df.hashposition.replace("C",1)
df.hashposition=df.hashposition.replace("R",2)
df['ispress'] = np.where((df['pff_PRESS'].isna()) , 0, 1)
df['isshotgun'] = np.where((df['pff_SHOTGUN'].isna()) , 0, 1)
df['isemptybackfield'] = np.where((df['pff_RBALIGNMENT']=="EMPTY") , 1, 0)
df['isrush'] = np.where((df['pff_RUNPASS'] == 'R'), 1, 0)
df['teleft'] = np.where((df['pff_TEALIGNMENT'].str[0]=="L")|(df['pff_TEALIGNMENT'].str[2]=="L")|(df['pff_TEALIGNMENT'].str[4]=="L")|(df['pff_TEALIGNMENT'].str[6]=="L") , 1, 0)
df['teright'] = np.where((df['pff_TEALIGNMENT'].str[0]=="R")|(df['pff_TEALIGNMENT'].str[2]=="R")|(df['pff_TEALIGNMENT'].str[4]=="R")|(df['pff_TEALIGNMENT'].str[6]=="R") , 1, 0)
df['teboth']= np.where((df['teleft']== 1)&(df['teright']==1),1,0)
df = df[df.pff_DEFPERSONNEL != "10 Men"]
df = df[df.pff_OFFPERSONNEL != "10 Men"]
df['numberlinemen']=df['pff_DEFPERSONNEL'].astype(str).str[0]
df['numberlinebackers']=df['pff_DEFPERSONNEL'].astype(str).str[2]
df['numbersecondary']=df['pff_DEFPERSONNEL'].astype(str).str[4]
df['numberwidereceivers']=df['pff_OFFPERSONNEL'].astype(str).str[0]
df['numberrunningbacks']=df['pff_OFFPERSONNEL'].astype(str).str[4]
df['numbertightends']=df['pff_OFFPERSONNEL'].astype(str).str[6]
df['numberrushers']=df['pff_PASSRUSHPLAYERS'].astype(str).str[0]
df['numbercoverage']=df['pff_PASSCOVERAGEPLAYERS'].astype(str).str[0]
df['numberwrleft']=df['pff_OFFFORMATIONGROUP'].astype(str).str[0]
df['numberwrright']=df['pff_OFFFORMATIONGROUP'].astype(str).str[2]
df['numberpassblocking']=df['pff_PASSBLOCKING'].astype(str).str[0]
df['numberboxplayers']=df['pff_BOXPLAYERS'].astype(str).str[0]
df=df[pd.to_numeric(df['numberlinemen'], errors='coerce').notnull()]
df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
df = df[df.numberwidereceivers != "X"]
df = df[df.numberrunningbacks != "X"]
df = df[df.numbertightends != "X"]
df.numberrushers=df.numberrushers.replace("n",0)
df.numberpassblocking=df.numberpassblocking.replace("n",0)
df.numbercoverage=df.numbercoverage.replace("n",0)
df['pff_DRIVE'] = df['pff_DRIVE'].astype(np.int64)
df['pff_DRIVEPLAY'] = df['pff_DRIVEPLAY'].astype(np.int64)
df['pff_GAINLOSS'] = df['pff_GAINLOSS'].astype(np.int64)
df['numberlinemen'] = df['numberlinemen'].astype(np.int64)
df['numberlinebackers'] = df['numberlinebackers'].astype(np.int64)
df['numbersecondary'] = df['numbersecondary'].astype(np.int64)
df['numberwidereceivers'] = df['numberwidereceivers'].astype(np.int64)
df['numberrunningbacks'] = df['numberrunningbacks'].astype(np.int64)
df['numbertightends'] = df['numbertightends'].astype(np.int64)
df['numberrushers'] = df['numberrushers'].astype(np.int64)
df['numbercoverage'] = df['numbercoverage'].astype(np.int64)
df['numberwrleft'] = df['numberwrleft'].astype(np.int64)
df['numberwrright'] = df['numberwrright'].astype(np.int64)
df['numberpassblocking'] = df['numberpassblocking'].astype(np.int64)
df['numberboxplayers'] = df['numberboxplayers'].astype(np.int64)
df['fieldposition'] = np.where((df['pff_FIELDPOSITION'] >= 0),df['pff_FIELDPOSITION'] , df['pff_FIELDPOSITION']+100)
df['efficent'] = np.where((df['pff_DOWN'] == 1) & (df['pff_DISTANCE']*.4 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 2) & (df['pff_DISTANCE']*.5 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 3) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 4) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS']), 1, 0)
df['redzone'] = np.where((df['pff_FIELDPOSITION'] <= 20), 1, 0)
df['backedup'] = np.where((df['pff_FIELDPOSITION'] >= 80) , 1, 0)
df['blitz'] = np.where((df['numberrushers'] > 4), 1, 0)
df = df[df.numberrushers !=0]
Kansasdata = df.loc[:,presnapoffense]

df = pd.read_csv('FinalKUTest.csv')
df=df[df.pff_NOPLAY ==0]
df=df[df["pff_PENALTYYARDS"].isnull()]
df.pff_OFFPERSONNELBASIC.dropna()
df.pff_OFFPERSONNEL.dropna()
df.pff_DEFPERSONNEL.dropna()
df.pff_GAINLOSS.fillna(0,inplace=True)
df['shownisclosed'] = np.where((df['pff_MOFOCSHOWN'] == "C") , 1, 0)
df['playedisclosed'] = np.where((df['pff_MOFOCPLAYED'] == "C") , 1, 0)
df['isman'] = np.where((df['pff_PASSCOVERAGE']=="0")|(df['pff_PASSCOVERAGE']=='1')|(df['pff_PASSCOVERAGE']=='1D')|(df['pff_PASSCOVERAGE']=='2M')|(df['pff_PASSCOVERAGE']=='GL') , 1, 0)
df['hashposition'] = df['pff_HASH']
df.hashposition=df.hashposition.replace("L",0)
df.hashposition=df.hashposition.replace("C",1)
df.hashposition=df.hashposition.replace("R",2)
df['ispress'] = np.where((df['pff_PRESS'].isna()) , 0, 1)
df['isshotgun'] = np.where((df['pff_SHOTGUN'].isna()) , 0, 1)
df['isemptybackfield'] = np.where((df['pff_RBALIGNMENT']=="EMPTY") , 1, 0)
df['isrush'] = np.where((df['pff_RUNPASS'] == 'R'), 1, 0)
df['teleft'] = np.where((df['pff_TEALIGNMENT'].str[0]=="L")|(df['pff_TEALIGNMENT'].str[2]=="L")|(df['pff_TEALIGNMENT'].str[4]=="L")|(df['pff_TEALIGNMENT'].str[6]=="L") , 1, 0)
df['teright'] = np.where((df['pff_TEALIGNMENT'].str[0]=="R")|(df['pff_TEALIGNMENT'].str[2]=="R")|(df['pff_TEALIGNMENT'].str[4]=="R")|(df['pff_TEALIGNMENT'].str[6]=="R") , 1, 0)
df['teboth']= np.where((df['teleft']== 1)&(df['teright']==1),1,0)
df = df[df.pff_DEFPERSONNEL != "10 Men"]
df = df[df.pff_OFFPERSONNEL != "10 Men"]
df['numberlinemen']=df['pff_DEFPERSONNEL'].astype(str).str[0]
df['numberlinebackers']=df['pff_DEFPERSONNEL'].astype(str).str[2]
df['numbersecondary']=df['pff_DEFPERSONNEL'].astype(str).str[4]
df['numberwidereceivers']=df['pff_OFFPERSONNEL'].astype(str).str[0]
df['numberrunningbacks']=df['pff_OFFPERSONNEL'].astype(str).str[4]
df['numbertightends']=df['pff_OFFPERSONNEL'].astype(str).str[6]
df['numberrushers']=df['pff_PASSRUSHPLAYERS'].astype(str).str[0]
df['numbercoverage']=df['pff_PASSCOVERAGEPLAYERS'].astype(str).str[0]
df['numberwrleft']=df['pff_OFFFORMATIONGROUP'].astype(str).str[0]
df['numberwrright']=df['pff_OFFFORMATIONGROUP'].astype(str).str[2]
df['numberpassblocking']=df['pff_PASSBLOCKING'].astype(str).str[0]
df['numberboxplayers']=df['pff_BOXPLAYERS'].astype(str).str[0]
df=df[pd.to_numeric(df['numberlinemen'], errors='coerce').notnull()]
df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
df = df[df.numberwidereceivers != "X"]
df = df[df.numberrunningbacks != "X"]
df = df[df.numbertightends != "X"]
df.numberrushers=df.numberrushers.replace("n",0)
df.numberpassblocking=df.numberpassblocking.replace("n",0)
df.numbercoverage=df.numbercoverage.replace("n",0)
df['pff_DRIVE'] = df['pff_DRIVE'].astype(np.int64)
df['pff_DRIVEPLAY'] = df['pff_DRIVEPLAY'].astype(np.int64)
df['pff_GAINLOSS'] = df['pff_GAINLOSS'].astype(np.int64)
df['numberlinemen'] = df['numberlinemen'].astype(np.int64)
df['numberlinebackers'] = df['numberlinebackers'].astype(np.int64)
df['numbersecondary'] = df['numbersecondary'].astype(np.int64)
df['numberwidereceivers'] = df['numberwidereceivers'].astype(np.int64)
df['numberrunningbacks'] = df['numberrunningbacks'].astype(np.int64)
df['numbertightends'] = df['numbertightends'].astype(np.int64)
df['numberrushers'] = df['numberrushers'].astype(np.int64)
df['numbercoverage'] = df['numbercoverage'].astype(np.int64)
df['numberwrleft'] = df['numberwrleft'].astype(np.int64)
df['numberwrright'] = df['numberwrright'].astype(np.int64)
df['numberpassblocking'] = df['numberpassblocking'].astype(np.int64)
df['numberboxplayers'] = df['numberboxplayers'].astype(np.int64)
df['fieldposition'] = np.where((df['pff_FIELDPOSITION'] >= 0),df['pff_FIELDPOSITION'] , df['pff_FIELDPOSITION']+100)
df['efficent'] = np.where((df['pff_DOWN'] == 1) & (df['pff_DISTANCE']*.4 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 2) & (df['pff_DISTANCE']*.5 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 3) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 4) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS']), 1, 0)
df['redzone'] = np.where((df['pff_FIELDPOSITION'] <= 20), 1, 0)
df['backedup'] = np.where((df['pff_FIELDPOSITION'] >= 80) , 1, 0)
df['blitz'] = np.where((df['numberrushers'] > 4), 1, 0)
df = df[df.numberrushers !=0]
Kansastestdata = df.loc[:,presnapoffense]

# Creates the model
KansasX_train, KansasX_test = Kansasdata.drop('blitz', axis=1), Kansastestdata.drop('blitz', axis=1)
Kansasy_train, Kansasy_test = Kansasdata['blitz'], Kansastestdata['blitz']
Kansaslogmodel = LogisticRegression()
Kansaslogmodel.fit(KansasX_train, Kansasy_train)
Kansasy_pred = logmodel.predict(KansasX_test)
print('-------------------------KANSAS MODEL-----------------------------')
print('Classification Report:')
print('')
print(classification_report(Kansasy_test, Kansasy_pred))
print('MODEL ACCURACY:',classification_report(Kansasy_test, Kansasy_pred)[205:207],'%')
# Prints confusion matrix
print('------------------------------------------------------')
print('Confusion Matrix:')
print('')
print(confusion_matrix(Kansasy_test, Kansasy_pred))

-------------------------KANSAS MODEL-----------------------------
Classification Report:

              precision    recall  f1-score   support

           0       0.92      0.92      0.92        25
           1       0.00      0.00      0.00         2

    accuracy                           0.85        27
   macro avg       0.46      0.46      0.46        27
weighted avg       0.85      0.85      0.85        27

MODEL ACCURACY: 85 %
------------------------------------------------------
Confusion Matrix:

[[23  2]
 [ 2  0]]


C:\Users\2tann\AppData\Local\Temp\ipykernel_19216\2816740399.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
C:\Users\2tann\AppData\Local\Temp\ipykernel_19216\2816740399.py:107: FutureWarning: In a future version of pandas

In [9]:
# This part makes it so that you only look at the pass plays
df = pd.read_csv('FinalWVUTrain.csv')
df=df[df.pff_NOPLAY ==0]
df=df[df["pff_PENALTYYARDS"].isnull()]
df.pff_OFFPERSONNELBASIC.dropna()
df.pff_OFFPERSONNEL.dropna()
df.pff_DEFPERSONNEL.dropna()
df.pff_GAINLOSS.fillna(0,inplace=True)
df['shownisclosed'] = np.where((df['pff_MOFOCSHOWN'] == "C") , 1, 0)
df['playedisclosed'] = np.where((df['pff_MOFOCPLAYED'] == "C") , 1, 0)
df['isman'] = np.where((df['pff_PASSCOVERAGE']=="0")|(df['pff_PASSCOVERAGE']=='1')|(df['pff_PASSCOVERAGE']=='1D')|(df['pff_PASSCOVERAGE']=='2M')|(df['pff_PASSCOVERAGE']=='GL') , 1, 0)
df['hashposition'] = df['pff_HASH']
df.hashposition=df.hashposition.replace("L",0)
df.hashposition=df.hashposition.replace("C",1)
df.hashposition=df.hashposition.replace("R",2)
df['ispress'] = np.where((df['pff_PRESS'].isna()) , 0, 1)
df['isshotgun'] = np.where((df['pff_SHOTGUN'].isna()) , 0, 1)
df['isemptybackfield'] = np.where((df['pff_RBALIGNMENT']=="EMPTY") , 1, 0)
df['isrush'] = np.where((df['pff_RUNPASS'] == 'R'), 1, 0)
df['teleft'] = np.where((df['pff_TEALIGNMENT'].str[0]=="L")|(df['pff_TEALIGNMENT'].str[2]=="L")|(df['pff_TEALIGNMENT'].str[4]=="L")|(df['pff_TEALIGNMENT'].str[6]=="L") , 1, 0)
df['teright'] = np.where((df['pff_TEALIGNMENT'].str[0]=="R")|(df['pff_TEALIGNMENT'].str[2]=="R")|(df['pff_TEALIGNMENT'].str[4]=="R")|(df['pff_TEALIGNMENT'].str[6]=="R") , 1, 0)
df['teboth']= np.where((df['teleft']== 1)&(df['teright']==1),1,0)
df = df[df.pff_DEFPERSONNEL != "10 Men"]
df = df[df.pff_OFFPERSONNEL != "10 Men"]
df['numberlinemen']=df['pff_DEFPERSONNEL'].astype(str).str[0]
df['numberlinebackers']=df['pff_DEFPERSONNEL'].astype(str).str[2]
df['numbersecondary']=df['pff_DEFPERSONNEL'].astype(str).str[4]
df['numberwidereceivers']=df['pff_OFFPERSONNEL'].astype(str).str[0]
df['numberrunningbacks']=df['pff_OFFPERSONNEL'].astype(str).str[4]
df['numbertightends']=df['pff_OFFPERSONNEL'].astype(str).str[6]
df['numberrushers']=df['pff_PASSRUSHPLAYERS'].astype(str).str[0]
df['numbercoverage']=df['pff_PASSCOVERAGEPLAYERS'].astype(str).str[0]
df['numberwrleft']=df['pff_OFFFORMATIONGROUP'].astype(str).str[0]
df['numberwrright']=df['pff_OFFFORMATIONGROUP'].astype(str).str[2]
df['numberpassblocking']=df['pff_PASSBLOCKING'].astype(str).str[0]
df['numberboxplayers']=df['pff_BOXPLAYERS'].astype(str).str[0]
df=df[pd.to_numeric(df['numberlinemen'], errors='coerce').notnull()]
df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
df = df[df.numberwidereceivers != "X"]
df = df[df.numberrunningbacks != "X"]
df = df[df.numbertightends != "X"]
df.numberrushers=df.numberrushers.replace("n",0)
df.numberpassblocking=df.numberpassblocking.replace("n",0)
df.numbercoverage=df.numbercoverage.replace("n",0)
df['pff_DRIVE'] = df['pff_DRIVE'].astype(np.int64)
df['pff_DRIVEPLAY'] = df['pff_DRIVEPLAY'].astype(np.int64)
df['pff_GAINLOSS'] = df['pff_GAINLOSS'].astype(np.int64)
df['numberlinemen'] = df['numberlinemen'].astype(np.int64)
df['numberlinebackers'] = df['numberlinebackers'].astype(np.int64)
df['numbersecondary'] = df['numbersecondary'].astype(np.int64)
df['numberwidereceivers'] = df['numberwidereceivers'].astype(np.int64)
df['numberrunningbacks'] = df['numberrunningbacks'].astype(np.int64)
df['numbertightends'] = df['numbertightends'].astype(np.int64)
df['numberrushers'] = df['numberrushers'].astype(np.int64)
df['numbercoverage'] = df['numbercoverage'].astype(np.int64)
df['numberwrleft'] = df['numberwrleft'].astype(np.int64)
df['numberwrright'] = df['numberwrright'].astype(np.int64)
df['numberpassblocking'] = df['numberpassblocking'].astype(np.int64)
df['numberboxplayers'] = df['numberboxplayers'].astype(np.int64)
df['fieldposition'] = np.where((df['pff_FIELDPOSITION'] >= 0),df['pff_FIELDPOSITION'] , df['pff_FIELDPOSITION']+100)
df['efficent'] = np.where((df['pff_DOWN'] == 1) & (df['pff_DISTANCE']*.4 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 2) & (df['pff_DISTANCE']*.5 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 3) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 4) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS']), 1, 0)
df['redzone'] = np.where((df['pff_FIELDPOSITION'] <= 20), 1, 0)
df['backedup'] = np.where((df['pff_FIELDPOSITION'] >= 80) , 1, 0)
df['blitz'] = np.where((df['numberrushers'] > 4), 1, 0)
df = df[df.numberrushers !=0]
WVUdata = df.loc[:,presnapoffense]

df = pd.read_csv('FinalWVUTest.csv')
df=df[df.pff_NOPLAY ==0]
df=df[df["pff_PENALTYYARDS"].isnull()]
df.pff_OFFPERSONNELBASIC.dropna()
df.pff_OFFPERSONNEL.dropna()
df.pff_DEFPERSONNEL.dropna()
df.pff_GAINLOSS.fillna(0,inplace=True)
df['shownisclosed'] = np.where((df['pff_MOFOCSHOWN'] == "C") , 1, 0)
df['playedisclosed'] = np.where((df['pff_MOFOCPLAYED'] == "C") , 1, 0)
df['isman'] = np.where((df['pff_PASSCOVERAGE']=="0")|(df['pff_PASSCOVERAGE']=='1')|(df['pff_PASSCOVERAGE']=='1D')|(df['pff_PASSCOVERAGE']=='2M')|(df['pff_PASSCOVERAGE']=='GL') , 1, 0)
df['hashposition'] = df['pff_HASH']
df.hashposition=df.hashposition.replace("L",0)
df.hashposition=df.hashposition.replace("C",1)
df.hashposition=df.hashposition.replace("R",2)
df['ispress'] = np.where((df['pff_PRESS'].isna()) , 0, 1)
df['isshotgun'] = np.where((df['pff_SHOTGUN'].isna()) , 0, 1)
df['isemptybackfield'] = np.where((df['pff_RBALIGNMENT']=="EMPTY") , 1, 0)
df['isrush'] = np.where((df['pff_RUNPASS'] == 'R'), 1, 0)
df['teleft'] = np.where((df['pff_TEALIGNMENT'].str[0]=="L")|(df['pff_TEALIGNMENT'].str[2]=="L")|(df['pff_TEALIGNMENT'].str[4]=="L")|(df['pff_TEALIGNMENT'].str[6]=="L") , 1, 0)
df['teright'] = np.where((df['pff_TEALIGNMENT'].str[0]=="R")|(df['pff_TEALIGNMENT'].str[2]=="R")|(df['pff_TEALIGNMENT'].str[4]=="R")|(df['pff_TEALIGNMENT'].str[6]=="R") , 1, 0)
df['teboth']= np.where((df['teleft']== 1)&(df['teright']==1),1,0)
df = df[df.pff_DEFPERSONNEL != "10 Men"]
df = df[df.pff_OFFPERSONNEL != "10 Men"]
df['numberlinemen']=df['pff_DEFPERSONNEL'].astype(str).str[0]
df['numberlinebackers']=df['pff_DEFPERSONNEL'].astype(str).str[2]
df['numbersecondary']=df['pff_DEFPERSONNEL'].astype(str).str[4]
df['numberwidereceivers']=df['pff_OFFPERSONNEL'].astype(str).str[0]
df['numberrunningbacks']=df['pff_OFFPERSONNEL'].astype(str).str[4]
df['numbertightends']=df['pff_OFFPERSONNEL'].astype(str).str[6]
df['numberrushers']=df['pff_PASSRUSHPLAYERS'].astype(str).str[0]
df['numbercoverage']=df['pff_PASSCOVERAGEPLAYERS'].astype(str).str[0]
df['numberwrleft']=df['pff_OFFFORMATIONGROUP'].astype(str).str[0]
df['numberwrright']=df['pff_OFFFORMATIONGROUP'].astype(str).str[2]
df['numberpassblocking']=df['pff_PASSBLOCKING'].astype(str).str[0]
df['numberboxplayers']=df['pff_BOXPLAYERS'].astype(str).str[0]
df=df[pd.to_numeric(df['numberlinemen'], errors='coerce').notnull()]
df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
df = df[df.numberwidereceivers != "X"]
df = df[df.numberrunningbacks != "X"]
df = df[df.numbertightends != "X"]
df.numberrushers=df.numberrushers.replace("n",0)
df.numberpassblocking=df.numberpassblocking.replace("n",0)
df.numbercoverage=df.numbercoverage.replace("n",0)
df['pff_DRIVE'] = df['pff_DRIVE'].astype(np.int64)
df['pff_DRIVEPLAY'] = df['pff_DRIVEPLAY'].astype(np.int64)
df['pff_GAINLOSS'] = df['pff_GAINLOSS'].astype(np.int64)
df['numberlinemen'] = df['numberlinemen'].astype(np.int64)
df['numberlinebackers'] = df['numberlinebackers'].astype(np.int64)
df['numbersecondary'] = df['numbersecondary'].astype(np.int64)
df['numberwidereceivers'] = df['numberwidereceivers'].astype(np.int64)
df['numberrunningbacks'] = df['numberrunningbacks'].astype(np.int64)
df['numbertightends'] = df['numbertightends'].astype(np.int64)
df['numberrushers'] = df['numberrushers'].astype(np.int64)
df['numbercoverage'] = df['numbercoverage'].astype(np.int64)
df['numberwrleft'] = df['numberwrleft'].astype(np.int64)
df['numberwrright'] = df['numberwrright'].astype(np.int64)
df['numberpassblocking'] = df['numberpassblocking'].astype(np.int64)
df['numberboxplayers'] = df['numberboxplayers'].astype(np.int64)
df['fieldposition'] = np.where((df['pff_FIELDPOSITION'] >= 0),df['pff_FIELDPOSITION'] , df['pff_FIELDPOSITION']+100)
df['efficent'] = np.where((df['pff_DOWN'] == 1) & (df['pff_DISTANCE']*.4 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 2) & (df['pff_DISTANCE']*.5 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 3) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 4) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS']), 1, 0)
df['redzone'] = np.where((df['pff_FIELDPOSITION'] <= 20), 1, 0)
df['backedup'] = np.where((df['pff_FIELDPOSITION'] >= 80) , 1, 0)
df['blitz'] = np.where((df['numberrushers'] > 4), 1, 0)
df = df[df.numberrushers !=0]
WVUtestdata = df.loc[:,presnapoffense]

# Creates the model
WVUX_train, WVUX_test = WVUdata.drop('blitz', axis=1), WVUtestdata.drop('blitz', axis=1)
WVUy_train, WVUy_test = WVUdata['blitz'], WVUtestdata['blitz']
WVUlogmodel = LogisticRegression()
WVUlogmodel.fit(WVUX_train, WVUy_train)
WVUy_pred = logmodel.predict(WVUX_test)
print('-------------------------WEST VIRGINIA MODEL-----------------------------')
print('Classification Report:')
print('')
print(classification_report(WVUy_test, WVUy_pred))
print('')
print('MODEL ACCURACY:',classification_report(WVUy_test, WVUy_pred)[205:207],'%')
# Prints confusion matrix
print('------------------------------------------------------')
print('Confusion Matrix:')
print('')
print(confusion_matrix(WVUy_test, WVUy_pred))

-------------------------WEST VIRGINIA MODEL-----------------------------
Classification Report:

              precision    recall  f1-score   support

           0       0.84      0.89      0.86        18
           1       0.00      0.00      0.00         3

    accuracy                           0.76        21
   macro avg       0.42      0.44      0.43        21
weighted avg       0.72      0.76      0.74        21


MODEL ACCURACY: 76 %
------------------------------------------------------
Confusion Matrix:

[[16  2]
 [ 3  0]]


C:\Users\2tann\AppData\Local\Temp\ipykernel_19216\2338908933.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
C:\Users\2tann\AppData\Local\Temp\ipykernel_19216\2338908933.py:107: FutureWarning: In a future version of pandas

In [10]:
# This part makes it so that you only look at the pass plays
df = pd.read_csv('FinalTXTrain.csv')
df=df[df.pff_NOPLAY ==0]
df=df[df["pff_PENALTYYARDS"].isnull()]
df.pff_OFFPERSONNELBASIC.dropna()
df.pff_OFFPERSONNEL.dropna()
df.pff_DEFPERSONNEL.dropna()
df.pff_GAINLOSS.fillna(0,inplace=True)
df['shownisclosed'] = np.where((df['pff_MOFOCSHOWN'] == "C") , 1, 0)
df['playedisclosed'] = np.where((df['pff_MOFOCPLAYED'] == "C") , 1, 0)
df['isman'] = np.where((df['pff_PASSCOVERAGE']=="0")|(df['pff_PASSCOVERAGE']=='1')|(df['pff_PASSCOVERAGE']=='1D')|(df['pff_PASSCOVERAGE']=='2M')|(df['pff_PASSCOVERAGE']=='GL') , 1, 0)
df['hashposition'] = df['pff_HASH']
df.hashposition=df.hashposition.replace("L",0)
df.hashposition=df.hashposition.replace("C",1)
df.hashposition=df.hashposition.replace("R",2)
df['ispress'] = np.where((df['pff_PRESS'].isna()) , 0, 1)
df['isshotgun'] = np.where((df['pff_SHOTGUN'].isna()) , 0, 1)
df['isemptybackfield'] = np.where((df['pff_RBALIGNMENT']=="EMPTY") , 1, 0)
df['isrush'] = np.where((df['pff_RUNPASS'] == 'R'), 1, 0)
df['teleft'] = np.where((df['pff_TEALIGNMENT'].str[0]=="L")|(df['pff_TEALIGNMENT'].str[2]=="L")|(df['pff_TEALIGNMENT'].str[4]=="L")|(df['pff_TEALIGNMENT'].str[6]=="L") , 1, 0)
df['teright'] = np.where((df['pff_TEALIGNMENT'].str[0]=="R")|(df['pff_TEALIGNMENT'].str[2]=="R")|(df['pff_TEALIGNMENT'].str[4]=="R")|(df['pff_TEALIGNMENT'].str[6]=="R") , 1, 0)
df['teboth']= np.where((df['teleft']== 1)&(df['teright']==1),1,0)
df = df[df.pff_DEFPERSONNEL != "10 Men"]
df = df[df.pff_OFFPERSONNEL != "10 Men"]
df['numberlinemen']=df['pff_DEFPERSONNEL'].astype(str).str[0]
df['numberlinebackers']=df['pff_DEFPERSONNEL'].astype(str).str[2]
df['numbersecondary']=df['pff_DEFPERSONNEL'].astype(str).str[4]
df['numberwidereceivers']=df['pff_OFFPERSONNEL'].astype(str).str[0]
df['numberrunningbacks']=df['pff_OFFPERSONNEL'].astype(str).str[4]
df['numbertightends']=df['pff_OFFPERSONNEL'].astype(str).str[6]
df['numberrushers']=df['pff_PASSRUSHPLAYERS'].astype(str).str[0]
df['numbercoverage']=df['pff_PASSCOVERAGEPLAYERS'].astype(str).str[0]
df['numberwrleft']=df['pff_OFFFORMATIONGROUP'].astype(str).str[0]
df['numberwrright']=df['pff_OFFFORMATIONGROUP'].astype(str).str[2]
df['numberpassblocking']=df['pff_PASSBLOCKING'].astype(str).str[0]
df['numberboxplayers']=df['pff_BOXPLAYERS'].astype(str).str[0]
df=df[pd.to_numeric(df['numberlinemen'], errors='coerce').notnull()]
df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
df = df[df.numberwidereceivers != "X"]
df = df[df.numberrunningbacks != "X"]
df = df[df.numbertightends != "X"]
df.numberrushers=df.numberrushers.replace("n",0)
df.numberpassblocking=df.numberpassblocking.replace("n",0)
df.numbercoverage=df.numbercoverage.replace("n",0)
df['pff_DRIVE'] = df['pff_DRIVE'].astype(np.int64)
df['pff_DRIVEPLAY'] = df['pff_DRIVEPLAY'].astype(np.int64)
df['pff_GAINLOSS'] = df['pff_GAINLOSS'].astype(np.int64)
df['numberlinemen'] = df['numberlinemen'].astype(np.int64)
df['numberlinebackers'] = df['numberlinebackers'].astype(np.int64)
df['numbersecondary'] = df['numbersecondary'].astype(np.int64)
df['numberwidereceivers'] = df['numberwidereceivers'].astype(np.int64)
df['numberrunningbacks'] = df['numberrunningbacks'].astype(np.int64)
df['numbertightends'] = df['numbertightends'].astype(np.int64)
df['numberrushers'] = df['numberrushers'].astype(np.int64)
df['numbercoverage'] = df['numbercoverage'].astype(np.int64)
df['numberwrleft'] = df['numberwrleft'].astype(np.int64)
df['numberwrright'] = df['numberwrright'].astype(np.int64)
df['numberpassblocking'] = df['numberpassblocking'].astype(np.int64)
df['numberboxplayers'] = df['numberboxplayers'].astype(np.int64)
df['fieldposition'] = np.where((df['pff_FIELDPOSITION'] >= 0),df['pff_FIELDPOSITION'] , df['pff_FIELDPOSITION']+100)
df['efficent'] = np.where((df['pff_DOWN'] == 1) & (df['pff_DISTANCE']*.4 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 2) & (df['pff_DISTANCE']*.5 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 3) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 4) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS']), 1, 0)
df['redzone'] = np.where((df['pff_FIELDPOSITION'] <= 20), 1, 0)
df['backedup'] = np.where((df['pff_FIELDPOSITION'] >= 80) , 1, 0)
df['blitz'] = np.where((df['numberrushers'] > 4), 1, 0)
df = df[df.numberrushers !=0]
TXdata = df.loc[:,presnapoffense]

df = pd.read_csv('FinalTXTest.csv')
df=df[df.pff_NOPLAY ==0]
df=df[df["pff_PENALTYYARDS"].isnull()]
df.pff_OFFPERSONNELBASIC.dropna()
df.pff_OFFPERSONNEL.dropna()
df.pff_DEFPERSONNEL.dropna()
df.pff_GAINLOSS.fillna(0,inplace=True)
df['shownisclosed'] = np.where((df['pff_MOFOCSHOWN'] == "C") , 1, 0)
df['playedisclosed'] = np.where((df['pff_MOFOCPLAYED'] == "C") , 1, 0)
df['isman'] = np.where((df['pff_PASSCOVERAGE']=="0")|(df['pff_PASSCOVERAGE']=='1')|(df['pff_PASSCOVERAGE']=='1D')|(df['pff_PASSCOVERAGE']=='2M')|(df['pff_PASSCOVERAGE']=='GL') , 1, 0)
df['hashposition'] = df['pff_HASH']
df.hashposition=df.hashposition.replace("L",0)
df.hashposition=df.hashposition.replace("C",1)
df.hashposition=df.hashposition.replace("R",2)
df['ispress'] = np.where((df['pff_PRESS'].isna()) , 0, 1)
df['isshotgun'] = np.where((df['pff_SHOTGUN'].isna()) , 0, 1)
df['isemptybackfield'] = np.where((df['pff_RBALIGNMENT']=="EMPTY") , 1, 0)
df['isrush'] = np.where((df['pff_RUNPASS'] == 'R'), 1, 0)
df['teleft'] = np.where((df['pff_TEALIGNMENT'].str[0]=="L")|(df['pff_TEALIGNMENT'].str[2]=="L")|(df['pff_TEALIGNMENT'].str[4]=="L")|(df['pff_TEALIGNMENT'].str[6]=="L") , 1, 0)
df['teright'] = np.where((df['pff_TEALIGNMENT'].str[0]=="R")|(df['pff_TEALIGNMENT'].str[2]=="R")|(df['pff_TEALIGNMENT'].str[4]=="R")|(df['pff_TEALIGNMENT'].str[6]=="R") , 1, 0)
df['teboth']= np.where((df['teleft']== 1)&(df['teright']==1),1,0)
df = df[df.pff_DEFPERSONNEL != "10 Men"]
df = df[df.pff_OFFPERSONNEL != "10 Men"]
df['numberlinemen']=df['pff_DEFPERSONNEL'].astype(str).str[0]
df['numberlinebackers']=df['pff_DEFPERSONNEL'].astype(str).str[2]
df['numbersecondary']=df['pff_DEFPERSONNEL'].astype(str).str[4]
df['numberwidereceivers']=df['pff_OFFPERSONNEL'].astype(str).str[0]
df['numberrunningbacks']=df['pff_OFFPERSONNEL'].astype(str).str[4]
df['numbertightends']=df['pff_OFFPERSONNEL'].astype(str).str[6]
df['numberrushers']=df['pff_PASSRUSHPLAYERS'].astype(str).str[0]
df['numbercoverage']=df['pff_PASSCOVERAGEPLAYERS'].astype(str).str[0]
df['numberwrleft']=df['pff_OFFFORMATIONGROUP'].astype(str).str[0]
df['numberwrright']=df['pff_OFFFORMATIONGROUP'].astype(str).str[2]
df['numberpassblocking']=df['pff_PASSBLOCKING'].astype(str).str[0]
df['numberboxplayers']=df['pff_BOXPLAYERS'].astype(str).str[0]
df=df[pd.to_numeric(df['numberlinemen'], errors='coerce').notnull()]
df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
df = df[df.numberwidereceivers != "X"]
df = df[df.numberrunningbacks != "X"]
df = df[df.numbertightends != "X"]
df.numberrushers=df.numberrushers.replace("n",0)
df.numberpassblocking=df.numberpassblocking.replace("n",0)
df.numbercoverage=df.numbercoverage.replace("n",0)
df['pff_DRIVE'] = df['pff_DRIVE'].astype(np.int64)
df['pff_DRIVEPLAY'] = df['pff_DRIVEPLAY'].astype(np.int64)
df['pff_GAINLOSS'] = df['pff_GAINLOSS'].astype(np.int64)
df['numberlinemen'] = df['numberlinemen'].astype(np.int64)
df['numberlinebackers'] = df['numberlinebackers'].astype(np.int64)
df['numbersecondary'] = df['numbersecondary'].astype(np.int64)
df['numberwidereceivers'] = df['numberwidereceivers'].astype(np.int64)
df['numberrunningbacks'] = df['numberrunningbacks'].astype(np.int64)
df['numbertightends'] = df['numbertightends'].astype(np.int64)
df['numberrushers'] = df['numberrushers'].astype(np.int64)
df['numbercoverage'] = df['numbercoverage'].astype(np.int64)
df['numberwrleft'] = df['numberwrleft'].astype(np.int64)
df['numberwrright'] = df['numberwrright'].astype(np.int64)
df['numberpassblocking'] = df['numberpassblocking'].astype(np.int64)
df['numberboxplayers'] = df['numberboxplayers'].astype(np.int64)
df['fieldposition'] = np.where((df['pff_FIELDPOSITION'] >= 0),df['pff_FIELDPOSITION'] , df['pff_FIELDPOSITION']+100)
df['efficent'] = np.where((df['pff_DOWN'] == 1) & (df['pff_DISTANCE']*.4 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 2) & (df['pff_DISTANCE']*.5 <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 3) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS'])|
    (df['pff_DOWN'] == 4) & (df['pff_DISTANCE'] <= df['pff_GAINLOSS']), 1, 0)
df['redzone'] = np.where((df['pff_FIELDPOSITION'] <= 20), 1, 0)
df['backedup'] = np.where((df['pff_FIELDPOSITION'] >= 80) , 1, 0)
df['blitz'] = np.where((df['numberrushers'] > 4), 1, 0)
df = df[df.numberrushers !=0]
TXtestdata = df.loc[:,presnapoffense]

# Creates the model
TXX_train, TXX_test = TXdata.drop('blitz', axis=1), TXtestdata.drop('blitz', axis=1)
TXy_train, TXy_test = TXdata['blitz'], TXtestdata['blitz']
TXlogmodel = LogisticRegression()
TXlogmodel.fit(TXX_train, TXy_train)
TXy_pred = logmodel.predict(TXX_test)
print('-------------------------TEXAS MODEL-----------------------------')
print('Classification Report:')
print('')
print(classification_report(TXy_test, TXy_pred))
print('')
print('MODEL ACCURACY:',classification_report(TXy_test, TXy_pred)[205:207],'%')
# Prints confusion matrix
print('------------------------------------------------------')
print('Confusion Matrix:')
print('')
print(confusion_matrix(TXy_test, TXy_pred))

-------------------------TEXAS MODEL-----------------------------
Classification Report:

              precision    recall  f1-score   support

           0       0.92      0.92      0.92        12
           1       0.67      0.67      0.67         3

    accuracy                           0.87        15
   macro avg       0.79      0.79      0.79        15
weighted avg       0.87      0.87      0.87        15


MODEL ACCURACY: 87 %
------------------------------------------------------
Confusion Matrix:

[[11  1]
 [ 1  2]]


C:\Users\2tann\AppData\Local\Temp\ipykernel_19216\3834380604.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['pff_PLAYID','pff_WEEK','pff_GSISGAMEKEY','pff_GSISPLAYID','pff_QUARTER','pff_DRIVE','pff_DRIVEPLAY','pff_DOWN','pff_GAINLOSS','pff_TIMETOPRESSURE','pff_TIMETOTHROW','pff_DISTANCE','pff_FIELDPOSITION','pff_BLITZDOG','pff_OFFFORMATIONUNBALANCED','pff_STUNT','pff_SCREEN','pff_RUNPASSOPTION','shownisclosed','playedisclosed','isman','numberlinemen','numberlinebackers','numbersecondary','numberwidereceivers','numberrunningbacks','numbertightends','numberrushers','numbercoverage','numberboxplayers','hashposition','numberwrleft','numberwrright','numberpassblocking','ispress','isemptybackfield','isshotgun','teleft','teright','teboth']), 1, inplace=True)
C:\Users\2tann\AppData\Local\Temp\ipykernel_19216\3834380604.py:107: FutureWarning: In a future version of pandas